In [1]:
import numpy as np
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.datasets import CIFAR10,ImageFolder
from torch.utils.data import DataLoader
import torch
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from tqdm import tqdm

In [2]:
#Load Dataset


#set the transformation

transformation = transforms.Compose([
    
    transforms.Resize(64),
    transforms.ToTensor(),
  
])

dataset = CIFAR10(root='data/', download=True, transform=transformation)
test_dataset = CIFAR10(root='data/', train=False, transform=transformation)

Files already downloaded and verified


In [3]:
classes = dataset.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
torch.manual_seed(43)  #Reason behind setting random seed is that when you guys run this you will get the same split
val_size = 5000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(45000, 5000)

In [21]:
batch_size = 32

train_loader = DataLoader(train_ds, 16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, 16, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, 16, num_workers=4, pin_memory=True)

In [6]:
for images, _ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))

    break

KeyboardInterrupt: 

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
class Cifar10_Model(nn.Module):
    def __init__(self,number_of_classes):
        super().__init__()
        
        self.convo_Block_1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=(2,2))
            
        )
        
        self.convo_Block_2 = nn.Sequential(
            
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=(2,2))
            
        )
        
        self.linear_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64*16*16,out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024,out_features=10)
        )
        
    def forward(self,x):
        x = self.convo_Block_1(x)
        x = self.convo_Block_2(x)
        x = self.linear_layer(x)
        
        return x

In [8]:
model = Cifar10_Model(10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer =torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 10
val_acc = []
train_acc =[]
train_loss = []
val_loss = []
for epoch in range(epochs):
    for images,labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        y_pred = model(images)
        loss = loss_fn(y_pred,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _,train_predection = torch.max(y_pred.data,1)
        train_acc.append((train_predection == labels).sum().item() / train_predection.size(0))
    
    for val_img,val_labels in val_loader:
        val_labels = val_labels.to(device)
        val_img = val_img.to(device)
        
        val_output = model(val_img)
        
        val_loss = loss_fn(val_output,val_labels)
        _, predicted = torch.max(val_output.data, 1)
        val_acc.append((predicted == val_labels).sum().item() / predicted.size(0))
        
    print("Validation acc",np.mean(val_acc),"Train Acc",np.mean(train_acc))
        
        

        

100%|██████████| 2813/2813 [01:00<00:00, 46.58it/s] 


Validation acc 0.6934904153354633 Train Acc 0.9149040170636331


100%|██████████| 2813/2813 [00:50<00:00, 55.47it/s] 


Validation acc 0.6958865814696485 Train Acc 0.944365446142908


100%|██████████| 2813/2813 [01:10<00:00, 39.68it/s] 


...

100%|██████████| 2813/2813 [00:47<00:00, 59.16it/s] 


Validation acc 0.7169778354632588 Train Acc 0.9847943698897974


100%|██████████| 2813/2813 [00:40<00:00, 69.43it/s] 


Validation acc 0.7186057862974796 Train Acc 0.9864838843464866


100%|██████████| 2813/2813 [00:34<00:00, 81.96it/s] 


In [10]:
predicted

tensor([9, 3, 9, 5, 6, 5, 5, 8], device='cuda:0')

In [13]:
x = val_labels == predicted

In [15]:
x.sum().item()

6

In [17]:
predicted.size()

torch.Size([8])

In [18]:
len(predicted)

8

In [19]:
val_output

tensor([[  5.7624,  -0.6303,   2.9165,  -6.9882,  -4.1340,  -3.4629,  -8.0038,
          -0.1856,  -0.5537,  12.7286],
        [ -4.1578,  -3.5790,  -1.5399,   5.1986,   0.4688,   3.8025,   5.1652,
          -0.7878,  -2.5830,  -1.4828],
        [ -0.7294,   8.6027,  -3.0472,  -6.2162,  -4.7080,  -4.4611,  -0.0939,
          -4.2562,   5.9749,  11.3380],
        [ -1.8325,  -5.0839,   4.8998,  10.9626,  -9.9602,  16.5845,  -1.8353,
          -3.1021,  -7.5864,  -1.7919],
        [-13.6344,  -3.9467,   3.0395,   3.0390,   6.4264,   5.1888,  18.7426,
           1.4509, -10.3563,  -4.4836],
        [ -3.1506,  -6.6790,   2.2943,   7.6049,   7.9275,   8.2018,   1.0803,
          -0.2412,  -7.2047,  -7.3265],
        [ -3.3364,  -6.1666,   3.3819,   3.4862,   1.7722,   6.4218,   3.1505,
           3.2412,  -7.0934,  -4.0807],
        [ -2.9936,  -3.6492,  -2.6622,  -2.7206,  -4.2868,   0.3550,  -4.8570,
          -0.9790,  14.7663,   3.9148]], device='cuda:0',
       grad_fn=<AddmmBackward0

In [20]:
labels

tensor([2, 6, 9, 2, 1, 4, 4, 8], device='cuda:0')